In [2]:
# Few-shot learning is a technique that allows you to train a model on a small amount of data.
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(
	model_name="gpt-3.5-turbo",
	temperature=0.1, 
	streaming=True, 
	callbacks=[StreamingStdOutCallbackHandler()]
)
	

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = """
  Human: {question}
	AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_prompt)

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt, 
	examples=examples,
	suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country": "Finland"
})

AI: 
        Here is what I know:
        
        Capital: Helsinki
        Language: Finnish, Swedish
        Food: Rye bread, fish, berries
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        \n        Capital: Helsinki\n        Language: Finnish, Swedish\n        Food: Rye bread, fish, berries\n        Currency: Euro')

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = ChatPromptTemplate.from_messages([
	("human", "What do you know about {country}?"),
	("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a geography expert."),
	example_prompt, # Put the few-shot prompt here
	("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({"country": "Turkey"})


        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira
        

AIMessageChunk(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira\n        ')

In [9]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")


'Human: What do you know about France?\nAI: \n        Here is what I know:\n        \n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [14]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self, input_examples):
        from random import choice
        return [choice(self.examples)]

example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [21]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
a = prompt.format(country="Germany")

yaml_prompt = load_prompt("./prompt.yaml")
b = yaml_prompt.format(food="Kebab")

a, b

('What is the capital of Germany?', 'What are the ingredients in Kebab?')

In [25]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrg! That is a secret!! Arg!!",
    question="What is your fav food?"
)



'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrg! That is a secret!! Arg!!\n\n                              \n    \n    Start now!\n\n    Human: What is your fav food?\n    You:\n\n'

In [27]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(temperature=0.1)

chat.predict("How to make a Kimchi-fried rice?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to make a Kimchi-fried rice?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [7.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Kimchi-fried rice, you will need the following ingredients:\n\n- 2 cups cooked rice\n- 1 cup chopped kimchi\n- 2 tablespoons kimchi juice\n- 1 tablespoon soy sauce\n- 1 tablespoon sesame oil\n- 1 tablespoon vegetable oil\n- 1/2 cup diced onion\n- 1/2 cup diced carrots\n- 1/2 cup diced bell pepper\n- 2 cloves garlic, minced\n- 2 green onions, chopped\n- 2 eggs\n- Salt and pepper to taste\n\nInstructions:\n\n1. Heat vegetable oil in a large skillet over medium heat. Add diced onion, carrots, bell pepper, and garlic. Cook until vegetables are tender, about 5 minutes.\n\n2. Add chopped kimchi and cook for another 2-3 minutes.\n\n3. Push the vegetables and kimchi to one side of the skillet and add the cooked rice to the other side. Break up

'To make Kimchi-fried rice, you will need the following ingredients:\n\n- 2 cups cooked rice\n- 1 cup chopped kimchi\n- 2 tablespoons kimchi juice\n- 1 tablespoon soy sauce\n- 1 tablespoon sesame oil\n- 1 tablespoon vegetable oil\n- 1/2 cup diced onion\n- 1/2 cup diced carrots\n- 1/2 cup diced bell pepper\n- 2 cloves garlic, minced\n- 2 green onions, chopped\n- 2 eggs\n- Salt and pepper to taste\n\nInstructions:\n\n1. Heat vegetable oil in a large skillet over medium heat. Add diced onion, carrots, bell pepper, and garlic. Cook until vegetables are tender, about 5 minutes.\n\n2. Add chopped kimchi and cook for another 2-3 minutes.\n\n3. Push the vegetables and kimchi to one side of the skillet and add the cooked rice to the other side. Break up any clumps of rice with a spatula.\n\n4. Add soy sauce, kimchi juice, and sesame oil to the rice. Mix everything together and cook for another 2-3 minutes.\n\n5. Push the rice to one side of the skillet and crack the eggs into the empty side. Sc

In [28]:
chat.predict("How to make a Kimchi-fried rice?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to make a Kimchi-fried rice?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Kimchi-fried rice, you will need the following ingredients:\n\n- 2 cups cooked rice\n- 1 cup chopped kimchi\n- 2 tablespoons kimchi juice\n- 1 tablespoon soy sauce\n- 1 tablespoon sesame oil\n- 1 tablespoon vegetable oil\n- 1/2 cup diced onion\n- 1/2 cup diced carrots\n- 1/2 cup diced bell pepper\n- 2 cloves garlic, minced\n- 2 green onions, chopped\n- 2 eggs\n- Salt and pepper to taste\n\nInstructions:\n\n1. Heat vegetable oil in a large skillet over medium heat. Add diced onion, carrots, bell pepper, and garlic. Cook until vegetables are tender, about 5 minutes.\n\n2. Add chopped kimchi and cook for another 2-3 minutes.\n\n3. Push the vegetables and kimchi to one side of the skillet and add the cooked rice to the other side. Break up any 

'To make Kimchi-fried rice, you will need the following ingredients:\n\n- 2 cups cooked rice\n- 1 cup chopped kimchi\n- 2 tablespoons kimchi juice\n- 1 tablespoon soy sauce\n- 1 tablespoon sesame oil\n- 1 tablespoon vegetable oil\n- 1/2 cup diced onion\n- 1/2 cup diced carrots\n- 1/2 cup diced bell pepper\n- 2 cloves garlic, minced\n- 2 green onions, chopped\n- 2 eggs\n- Salt and pepper to taste\n\nInstructions:\n\n1. Heat vegetable oil in a large skillet over medium heat. Add diced onion, carrots, bell pepper, and garlic. Cook until vegetables are tender, about 5 minutes.\n\n2. Add chopped kimchi and cook for another 2-3 minutes.\n\n3. Push the vegetables and kimchi to one side of the skillet and add the cooked rice to the other side. Break up any clumps of rice with a spatula.\n\n4. Add soy sauce, kimchi juice, and sesame oil to the rice. Mix everything together and cook for another 2-3 minutes.\n\n5. Push the rice to one side of the skillet and crack the eggs into the empty side. Sc

In [ ]:
f